# TP de Text Mining

In [13]:
import os.path as op
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from glob import glob
import re

In [14]:
# Load data
print("Loading dataset")
filenames_neg = sorted(glob(op.join('data', 'imdb1', 'neg', '*.txt'))) # Assumes that data are located in data/imdb1/neg
filenames_pos = sorted(glob(op.join('data', 'imdb1', 'pos', '*.txt'))) # Assumes that data are located in data/imdb1/neg
texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

Loading dataset
2000 documents


## 1) Compléter la fonction count_word

In [23]:
def count_words(texts):
    voc = {}
    wordidx = 0
    counts = np.zeros((len(texts), 0))
    for textidx,text  in enumerate(texts):
        #print(textidx," / ",len(texts), " documents\r" )
        text = re.sub('\W+', " ", text).lower().strip().split(" ") # Removes non-words characters and tokenize text
        for word in text:
                # If the word's already in the vocabulary dict, increment the count of the word in the counts matrix
                if word in voc: # for the current document
                    counts[textidx, voc[word]] += 1
                # Else add the new word to the vocabulary dict and set it's index in the counts matrix
                else:
                    voc[word] = wordidx 
                    counts = np.c_[counts, np.zeros(len(texts))] # Add a new column in counts for the new word
                    # increment the count in counts for the current word in the current document
                    counts[textidx, wordidx] += 1 
                    wordidx += 1 # Increment the index for the next new word
    return vocabulary, counts

## Test de la fonction

In [ ]:
vocabulary, X = count_words(texts)

0  /  2000  documents
1  /  2000  documents
2  /  2000  documents
3  /  2000  documents
4  /  2000  documents
5  /  2000  documents
6  /  2000  documents
7  /  2000  documents
8  /  2000  documents
9  /  2000  documents
10  /  2000  documents
11  /  2000  documents
12  /  2000  documents
13  /  2000  documents
14  /  2000  documents
15  /  2000  documents
16  /  2000  documents
17  /  2000  documents
18  /  2000  documents
19  /  2000  documents
20  /  2000  documents
21  /  2000  documents
22  /  2000  documents
23  /  2000  documents
24  /  2000  documents
25  /  2000  documents
26  /  2000  documents
27  /  2000  documents
28  /  2000  documents
29  /  2000  documents
30  /  2000  documents
31  /  2000  documents
32  /  2000  documents
33  /  2000  documents
34  /  2000  documents
35  /  2000  documents
36  /  2000  documents
37  /  2000  documents
38  /  2000  documents
39  /  2000  documents
40  /  2000  documents
41  /  2000  documents
42  /  2000  documents
43  /  2000  document